In [56]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from math import log
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [142]:
class Node():
    def __init__(self,feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
           
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [143]:
#https://github.com/Suji04/ML_from_Scratch/blob/master/decision%20tree%20classification.ipynb
#constrctor code derived from ^

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, X,Y, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        # keep recursing and building nodes
        # else return Node that has leaf value
       
        n_sample,n_feature = np.shape(X)
        
        if n_sample>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(X,Y)
            if best_split['info_gain'] > 0:
                left_subtree = self.build_tree(best_split["x_left"],best_split["y_left"] ,curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["x_right"],best_split["y_right"] ,curr_depth+1)
                # return decision node

                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])

            
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    
    def get_best_split(self, X,Y):
        ''' function to find the best split 
        
        ---------------------------------------
        returns: dict
        best_split{feature_index,threshold,dataset_left,dataset_right,info_gain}
        
        '''
        
        min_gain = np.NINF
        best_split = {}
        for col in range(np.shape(X)[1]):
            
            split_points = sorted(list(set(X[:,col]))) #take jth feature column and get unique values to split on
            
            for s in split_points:
                
                greater_split_idx, less_split_idx  = np.where(X[:,col] > s)[0] , np.where(X[:,col] <= s)[0]
                
                greater_y, lesser_y = Y[greater_split_idx], Y[less_split_idx] # greater and less than split datasets
                
                if len(greater_y)>0 and len(lesser_y)>0:
                    
                    running_gain = self.information_gain(Y,greater_y,lesser_y)

                    if running_gain > min_gain:
                        min_gain = running_gain

                        best_split['feature_index'] =  col
                        best_split['threshold']     =  s

                        best_split['x_right']  = X[greater_split_idx,:]
                        best_split['y_right']  = greater_y

                        best_split['x_left'] = X[less_split_idx,:]
                        best_split['y_left'] = lesser_y
                        best_split['info_gain'] = min_gain
     
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
       
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="gini"):
        ''' function to compute information gain '''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def gini_index(self, y):
        ''' function to compute gini index '''

        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    
    def entropy(self,arr):

        value,counts = np.unique(arr,return_counts = True)

        p = [len(np.where(arr == v)[0])/len(arr) for v in value]

        return sum([-p*log(p,2) for p in p])
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        self.root = self.build_tree(X,Y)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        
        

In [147]:
iris = load_iris()
X_train, X_test, Y_train, Y_test = train_test_split(iris['data'], iris['target'], \
                                                    test_size=.2)

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_2 <= 1.9 ? 0.3150595238095238
 left:0
 right:X_3 <= 1.7 ? 0.38957707415536774
  left:X_2 <= 4.9 ? 0.0641984416063443
    left:X_0 <= 4.9 ? 0.04759071980963725
        left:2
        right:1
    right:X_3 <= 1.5 ? 0.48
        left:2
        right:1
  right:X_2 <= 4.8 ? 0.02493074792243752
    left:1
    right:2


In [148]:
Y_pred = classifier.predict(X_train) 

accuracy_score(Y_train, Y_pred)

0.9916666666666667